In [1]:
import os


In [2]:
os.chdir('../')

In [3]:
pwd

'e:\\gemstone'

In [5]:
from src.logging.logger import logging
from src.exception.exception import CustomException
from src.utils.utils import read_yaml,create_dir
from src.constant.ymal_path import *
from pymongo import MongoClient

from dotenv import load_dotenv
load_dotenv()
url=os.getenv('url')
db=os.getenv('db')
collection=os.getenv('collection')

In [34]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    dir:Path
    train_data: Path
    test_data: Path
    raw_data : Path

In [35]:
class ConfigManager:
    def __init__(self,
                 config_file_path=Config_ymal_file_path,
                 prams_file_path=Param_ymal_file_path,
                 schema_file_path=Schema_ymal_file_path) -> None:
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(prams_file_path)
        self.schema=read_yaml(schema_file_path)

        create_dir([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_dir([config.dir])

        data_ingestion_config = DataIngestionConfig(
           dir=config.dir,
           train_data=config.train_data,
           test_data=config.test_data,
           raw_data=config.raw_data
        )

        return data_ingestion_config

In [15]:
from src.utils.utils import read_data_from_db
from sklearn.model_selection import train_test_split
import sys

In [36]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig) -> None:
        self.config=config

    def initiate_data_ingestion(self):
        try:
            logging.info('data ingestion has started')
            df=read_data_from_db(url=url,db=db,collection=collection)

            print(df.head())
           
            df.to_csv(self.config.raw_data)

            train_data,test_data=train_test_split(df,test_size=0.26,random_state=0)

            train_data.to_csv(self.config.train_data)

            test_data.to_csv(self.config.test_data)

            logging.info('Data ingestion completed')

            return (
                self.config.test_data,
                self.config.test_data
            )            
        except PermissionError as e:
            logging.info(f'Error cooured {str(e)}')
            raise CustomException(sys,e)        

In [37]:
try:
    config=ConfigManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.initiate_data_ingestion()
except Exception as e:
            logging.info(f'Error cooured {str(e)}')
            raise CustomException(sys,e)           

   id  carat        cut color clarity  depth  table     x     y     z  price
0   0   1.52    Premium     F     VS2   62.2   58.0  7.27  7.33  4.55  13619
1   1   2.03  Very Good     J     SI2   62.0   58.0  8.06  8.12  5.05  13387
2   2   0.70      Ideal     G     VS1   61.2   57.0  5.69  5.73  3.50   2772
3   3   0.32      Ideal     G     VS1   61.6   56.0  4.38  4.41  2.71    666
4   4   1.70    Premium     G     VS2   62.6   59.0  7.65  7.61  4.77  14453
   id  carat        cut color clarity  depth  table     x     y     z  price
0   0   1.52    Premium     F     VS2   62.2   58.0  7.27  7.33  4.55  13619
1   1   2.03  Very Good     J     SI2   62.0   58.0  8.06  8.12  5.05  13387
2   2   0.70      Ideal     G     VS1   61.2   57.0  5.69  5.73  3.50   2772
3   3   0.32      Ideal     G     VS1   61.6   56.0  4.38  4.41  2.71    666
4   4   1.70    Premium     G     VS2   62.6   59.0  7.65  7.61  4.77  14453
